In [1]:
# Import libraries and data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import itertools as itt
import pickle as pk

full_train = pd.read_parquet('../data/cleaned/all_features_train.parquet')
full_test = pd.read_parquet('../data/cleaned/all_features_test.parquet')

In [2]:
# Split test and train data sets into X, y_norm, and y_log
X_train, y_train_norm, y_train_log = (full_train.drop(['price', 'log(price_plus_0)'], axis = 1),
                                      full_train.price,
                                      full_train['log(price_plus_0)'])
X_test, y_test_norm, y_test_log = (full_test.drop(['price', 'log(price_plus_0)'], axis = 1),
                                      full_test.price,
                                      full_test['log(price_plus_0)'])

In [3]:
# Add columns of zeroes for each column present in X_train but not present in X_test
# zipcode_98050 was not present in X_test and was duplicated through one-hot-encoding and engineered interactions
# Since all of these columns represent dummy variables, this should not affect modeling whatsoever
# X_test and X_train need to have the same shape for sklearn LinearRegression()
def equalize_columns(df_1, df_2):
    col_to_add_1 = [x for x in df_2.columns if x not in df_1.columns]
    col_to_add_2 = [x for x in df_1.columns if x not in df_2.columns]
    
    if col_to_add_1 != []:
        for col in col_to_add_1:
            df_1[col] = 0
            
    if col_to_add_2 != []:
        for col in col_to_add_2:
            df_2[col] = 0
            
    df_1_col_order = df_1.columns.values.tolist()
    
    df_2.reindex(columns = df_1_col_order)
    

equalize_columns(X_train, X_test)

In [4]:
# Runs a linear regression fitted on train data, then predicts on test data using a specified column set
# Creates a pandas DataFrame containing the R^2 value of X_train, y_train
# as well as the R^2 value obtained from predicting on X_test using coefficients from fitting on X_train
# Also contains the fitted coefficient names/values/p-scores
# Returns the y_prediction from predicting on X_test and the pandas DataFrame with tabular information
def run_linear_regression(X_train, y_train, X_test, y_test, column_subset):
    lr = LinearRegression()
    lr.fit(X_train[column_subset], y_train)
    lr_sm = sm.OLS(y_train, sm.add_constant(X_train[column_subset])).fit()
    
    regression_frame = pd.concat([pd.DataFrame(zip(['Test R^2'], [lr.score(X_test[column_subset], y_test)])),
                                  pd.DataFrame(zip(['Train R^2'], [lr_sm.rsquared])),
                                  lr_sm.params.reset_index().rename(columns = {'index': 0, 0: 1})])
    regression_frame['p_values'] = [None, None] + lr_sm.pvalues.values.tolist()
    regression_frame.columns = ['param', 'coeff', 'p_values']
    
    y_pred = lr.predict(X_test[column_subset])
    
    return y_pred, regression_frame

In [5]:
# Checks if any string in a given list is a substring of given string
def substring_from_list_in_string(list_one, test_string):
    for string in list_one:
        if string in test_string:
            return True
    return False

In [6]:
# Cleans an array of symmetric interaction terms (a_x_b vs. b_x_a) and returns only unique features
def get_unique_interactions(features):
    features_unique = []
    for feature in features:
        if feature in features_unique:
            continue
        if '_x_' not in feature:
            features_unique.append(feature)
        else:
            split_feature = feature.split('_x_')
            split_feature.reverse()
            rejoined_feature = '_x_'.join(split_feature)
            if rejoined_feature in features_unique:
                continue
            else:
                features_unique.append(feature)
            
    return features_unique

In [7]:
# Defining base categories for further selection use
binaries = ['waterfront', 'greenbelt', 'nuisance']
ordinals = ['view', 'condition', 'grade']
discrete = ['bedrooms', 'bathrooms', 'floors', 'yr_built']
continuous = [x for x in X_train.columns if 'sqft' in x and '_x_' not in x and 'log' not in x]
zipcodes = [x for x in X_train.columns if 'zip' in x and '_x_' not in x]
log_continuous = [x for x in X_train.columns if 'log' in x and '_x_' not in x]
ordinal_ohes = [x for x in X_train.columns if ('view' in x or 'condition' in x or 'grade' in x)\
                and '_' in x and '_x_' not in x]
ohes = [x for x in X_train.columns if '_' in x and '_x_' not in x\
        and x not in set(itt.chain(binaries, ordinals, discrete, continuous,\
                                   log_continuous, zipcodes, ordinal_ohes))]
interactions = [x for x in X_train.columns if '_x_' in x]

In [8]:
# Testing that the shape and values of base categories matches all column values in X_train
# In other words, the above lists partition all of X_train
print(len(X_train.columns.values) == len(set(binaries + ordinals + discrete\
                                        + continuous + zipcodes + log_continuous\
                                        + ordinal_ohes + ohes + interactions)),
      set(X_train.columns.values) == set(binaries + ordinals + discrete\
                                   + continuous + zipcodes + log_continuous\
                                   + ordinal_ohes + ohes + interactions),
      sep = '\n')

True
True


In [9]:
# Simple linear regression on highest correlated column - sqft_living
# Test R^2 of 0.4434 is not very strong
simple_columns = ['sqft_living']
y_pred, df = run_linear_regression(X_train, y_train_norm, X_test, y_test_norm, simple_columns)
df.iloc[:2]

,param,coeff,p_values
0,Test R^2,0.443392,NaN
0,Train R^2,0.420277,NaN


In [10]:
# Create multi-linear regression baseline using all of the base categories
# Except: continuous variables not log transformed and ordinal variables encoded ordinally instead of as ohe
# Much better Test R^2: 0.7512
baseline_columns = binaries + ordinals + discrete + continuous + zipcodes + ohes
y_pred_1, df_1 = run_linear_regression(X_train, y_train_norm, X_test, y_test_norm, baseline_columns)

# Test baseline regression using log-transformed y (price)
# Test R^2 slightly worse, 0.7269
y_pred_2, df_2 = run_linear_regression(X_train, y_train_log, X_test, y_test_log, baseline_columns)

# Write y_pred and df to pickle objects for use in visualizations
# with open('../data/cleaned/baseline_regression_y_pred.pickle', 'wb') as f:
#     pk.dump(y_pred, f)
# with open('../data/cleaned/baseline_regression_coeff_frame.pickle', 'wb') as f:
#     pk.dump(df, f)

print(df_1.iloc[:2], df_2.iloc[:2], sep = '\n\n')

       param     coeff  p_values
0   Test R^2  0.751223       NaN
0  Train R^2  0.757472       NaN

       param     coeff  p_values
0   Test R^2  0.726913       NaN
0  Train R^2  0.719369       NaN


In [11]:
# Test variation of baseline regression using log-transformed continuous variables
# Test R^2 against norm y: 0.7298, Test R^2 against log y: 0.7286, roughly equal but worse than baseline
baseline_log_cont = binaries + ordinals + discrete + log_continuous + zipcodes + ohes
y_pred_1, df_1 = run_linear_regression(X_train, y_train_norm, X_test, y_test_norm, baseline_log_cont)
y_pred_2, df_2 = run_linear_regression(X_train, y_train_log, X_test, y_test_log, baseline_log_cont)

print(df_1.iloc[:2], df_2.iloc[:2], sep = '\n\n')

# # Write y_pred_2 and df_2 to pickle objects for use in visualizations
# with open('../data/cleaned/log_continuous_y_pred_on_test.pickle', 'wb') as f:
#     pk.dump(y_pred_2, f)
# with open('../data/cleaned/log_continuous_coeff_frame.pickle', 'wb') as f:
#     pk.dump(df_2, f)

       param     coeff  p_values
0   Test R^2  0.729757       NaN
0  Train R^2  0.736595       NaN

       param     coeff  p_values
0   Test R^2  0.728624       NaN
0  Train R^2  0.720479       NaN


In [12]:
# Test converting all ordinal columns except for grade to one-hot-encoded columns
# Performance not greatly improved over baseline model
ordinals_not_grade = [x for x in ordinals if 'grade' not in x]
ordinals_ohe_not_grade = [x for x in ordinal_ohes if 'grade' not in x]
X_col = list(set(baseline_columns) - set(ordinals_not_grade)) + ordinals_ohe_not_grade
y_pred_1, df_1 = run_linear_regression(X_train, y_train_norm, X_test, y_test_norm, X_col)
y_pred_2, df_2 = run_linear_regression(X_train, y_train_log, X_test, y_test_log, X_col)
print(df_1.iloc[:2], df_2.iloc[:2], sep = '\n\n')

       param     coeff  p_values
0   Test R^2  0.753132       NaN
0  Train R^2  0.761993       NaN

       param     coeff  p_values
0   Test R^2  0.727410       NaN
0  Train R^2  0.720576       NaN


In [13]:
# Same test as above but with log-transformed continuous variables
# Worse performance than before
X_col = list(set(baseline_log_cont) - set(ordinals_not_grade)) + ordinals_ohe_not_grade
y_pred_1, df_1 = run_linear_regression(X_train, y_train_norm, X_test, y_test_norm, X_col)
y_pred_2, df_2 = run_linear_regression(X_train, y_train_log, X_test, y_test_log, X_col)
print(df_1.iloc[:2], df_2.iloc[:2], sep = '\n\n')

       param     coeff  p_values
0   Test R^2  0.732101       NaN
0  Train R^2  0.741420       NaN

       param     coeff  p_values
0   Test R^2  0.729217       NaN
0  Train R^2  0.721780       NaN


In [14]:
# Zip code appeared to greatly affect all other independent variabels during EDA
# Test using zip interaction columns between binary and continuous variables
# Test R^2 against y norm: 0.7883, an improvement over the baseline model
# Test R^2 against y log has consistently performed worse compared to y norm, not worth calculating moving forward
binaries_and_continuous = binaries + continuous
zip_interactions_bc = [x for x in interactions if substring_from_list_in_string(binaries_and_continuous, x)\
                    and substring_from_list_in_string(zipcodes, x)\
                    and 'log' not in x]
X_col = baseline_columns + zip_interactions_bc
y_pred_1, df_1 = run_linear_regression(X_train, y_train_norm, X_test, y_test_norm, X_col)
y_pred_2, df_2 = run_linear_regression(X_train, y_train_log, X_test, y_test_log, X_col)
print(df_1.iloc[:2], df_2.iloc[:2], sep = '\n\n')

       param     coeff  p_values
0   Test R^2  0.788337       NaN
0  Train R^2  0.824724       NaN

       param     coeff  p_values
0   Test R^2  0.727210       NaN
0  Train R^2  0.742562       NaN


In [15]:
# Test using all zipcode interactions with baseline variables
# Test R^2 of 0.7844, worse than prior model
zipcode_interactions_baseline = [x for x in interactions if substring_from_list_in_string(baseline_columns, x)\
                    and substring_from_list_in_string(zipcodes, x)\
                    and not substring_from_list_in_string(ordinal_ohes, x)
                    and 'log' not in x]
X_col = get_unique_interactions(baseline_columns + zipcode_interactions_baseline)
y_pred_1, df_1 = run_linear_regression(X_train, y_train_norm, X_test, y_test_norm, X_col)
print(df_1.iloc[:2])

       param     coeff  p_values
0   Test R^2  0.784441       NaN
0  Train R^2  0.849847       NaN


In [16]:
# Try looking at numerical interactions containing at least one continuous or discrete feature
# This includes zipcode interactions involving continuous or discrete features
# Have to use get_unique_interactions to remove symmetrically selected interaction features
# Test R^2 on norm y: 0.7952, an improvement on all prior models!
cont_and_disc = continuous + discrete
numerical_interactions = [x for x in interactions if substring_from_list_in_string(cont_and_disc, x)\
                          and not substring_from_list_in_string(ordinal_ohes, x)\
                          and 'log' not in x]
X_col = get_unique_interactions(baseline_columns + numerical_interactions)
y_pred_1, df_1 = run_linear_regression(X_train, y_train_norm, X_test, y_test_norm, X_col)
print(df_1.iloc[:2])

       param     coeff  p_values
0   Test R^2  0.795226       NaN
0  Train R^2  0.831509       NaN


In [17]:
# Test using one-hot encoding for ordinal variables
# Test R^2 = 0.7900, good, but not as good as previous
baseline_ordohe = discrete + continuous + binaries + ordinal_ohes + ohes + zipcodes
zipcode_interactions_baseline_ordohe = [x for x in interactions if substring_from_list_in_string(baseline_ordohe, x)\
                    and substring_from_list_in_string(zipcodes, x)\
                    and not substring_from_list_in_string(ordinals, x)
                    and 'log' not in x]
X_col = get_unique_interactions(baseline_ordohe + zipcode_interactions_baseline_ordohe)
y_pred_1, df_1 = run_linear_regression(X_train, y_train_norm, X_test, y_test_norm, X_col)
print(df_1.iloc[:2])

       param     coeff  p_values
0   Test R^2  0.789665       NaN
0  Train R^2  0.847317       NaN


In [18]:
# Same test as above but with log-transformed continuous variables
# Test R^2 = 0.7908, still not as good as using numerical interactions
log_base_ordohe = discrete + log_continuous + binaries + ordinal_ohes + ohes + zipcodes
zipcode_interactions_log_base_ordohe = [x for x in interactions if substring_from_list_in_string(log_base_ordohe, x)\
                    and substring_from_list_in_string(zipcodes, x)\
                    and not substring_from_list_in_string(ordinals, x)
                    and 'log' not in x]
X_col = get_unique_interactions(log_base_ordohe + zipcode_interactions_log_base_ordohe)
y_pred_1, df_1 = run_linear_regression(X_train, y_train_norm, X_test, y_test_norm, X_col)
print(df_1.iloc[:2])

       param     coeff  p_values
0   Test R^2  0.790791       NaN
0  Train R^2  0.848229       NaN


In [19]:
# Try looking at only sqft_living and sqft_lot, as all other sqft categories are correlated
primary_sqft = ['sqft_living', 'sqft_lot']
primary_log_sqft = [x for x in log_continuous if substring_from_list_in_string(primary_sqft, x)]
primary_base = binaries + ordinals + discrete + primary_sqft + zipcodes + ohes
log_continuous_not_primary = [x for x in log_continuous if x not in primary_log_sqft]
log_primary_base = binaries + ordinals + discrete + primary_log_sqft + zipcodes + ohes

In [20]:
# Look at primary base with interactions containing at least one feature from primary_sqft or discrete
# Test R^2 = 0.8020, best result so far
prim_and_disc = primary_sqft + discrete
interactions_prim_disc = [x for x in interactions if substring_from_list_in_string(prim_and_disc, x)\
                          and not substring_from_list_in_string(ordinal_ohes, x)\
                          and 'log' not in x]
X_col = get_unique_interactions(primary_base + interactions_prim_disc)
y_pred_1, df_1 = run_linear_regression(X_train, y_train_norm, X_test, y_test_norm, X_col)
print(df_1.iloc[:2])

       param     coeff  p_values
0   Test R^2  0.801962       NaN
0  Train R^2  0.822836       NaN


In [21]:
# Try the same as above but using log-transformed variables
# Test R^2 = 0.7982, not bad, but not better than previous
log_prim_and_disc = primary_log_sqft + discrete
int_log = [x for x in interactions if substring_from_list_in_string(primary_log_sqft, x)\
          and not substring_from_list_in_string(ordinal_ohes, x)]
int_disc = [x for x in interactions if substring_from_list_in_string(discrete, x)\
          and not substring_from_list_in_string(ordinal_ohes, x)\
           and not substring_from_list_in_string(continuous, x)\
           and not substring_from_list_in_string(log_continuous_not_primary, x)]
X_col = get_unique_interactions(log_primary_base + int_log + int_disc)
y_pred_1, df_1 = run_linear_regression(X_train, y_train_norm, X_test, y_test_norm, X_col)
print(df_1.iloc[:2])

       param     coeff  p_values
0   Test R^2  0.798162       NaN
0  Train R^2  0.816124       NaN
